In [71]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn 
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras
import pathlib

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, keras:
    print(module.__name__, module.__version__)

2.2.0-rc3
sys.version_info(major=3, minor=8, micro=2, releaselevel='final', serial=0)
matplotlib 3.2.1
numpy 1.18.3
pandas 1.0.3
sklearn 0.22.2.post1
tensorflow.keras 2.3.0-tf


In [72]:
data_dir = '/home/share//fashion_image_label/'

In [73]:
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('/home/share/fashion_image_label')

In [74]:
for i in data_dir.glob('*'):
    print(i)

/home/share/fashion_image_label/fashion_train_jpg_60000.txt
/home/share/fashion_image_label/fashion_test_jpg_10000
/home/share/fashion_image_label/fashion_train_jpg_60000
/home/share/fashion_image_label/fashion_test_jpg_10000.txt


In [77]:
num_train_sample = len(list(data_dir.glob('fashion_train_jpg_60000/*')))
num_test_sample = len(list(data_dir.glob('fashion_test_jpg_10000/*')))

In [78]:
for i in list(data_dir.glob('fashion_train_jpg_60000/*'))[:5]:
    print(i)

/home/share/fashion_image_label/fashion_train_jpg_60000/23530_2.jpg
/home/share/fashion_image_label/fashion_train_jpg_60000/33464_6.jpg
/home/share/fashion_image_label/fashion_train_jpg_60000/52431_7.jpg
/home/share/fashion_image_label/fashion_train_jpg_60000/15171_2.jpg
/home/share/fashion_image_label/fashion_train_jpg_60000/9874_0.jpg


In [79]:
for i in data_dir.glob('*.txt'):
    print(i)

/home/share/fashion_image_label/fashion_train_jpg_60000.txt
/home/share/fashion_image_label/fashion_test_jpg_10000.txt


In [80]:
(data_dir/'fashion_train_jpg_60000.txt').open(encoding='utf-8').readlines()[:5]

['1_9.jpg 9\n', '2_0.jpg 0\n', '3_0.jpg 0\n', '4_3.jpg 3\n', '5_0.jpg 0\n']

In [81]:
label_info = (data_dir/'fashion_train_jpg_60000.txt').open(encoding='utf-8').readlines()

In [96]:
for i in label_info[:5]:
    print(i)

1_9.jpg 9

2_0.jpg 0

3_0.jpg 0

4_3.jpg 3

5_0.jpg 0



In [83]:
labels = [item.split()[-1] for item in label_info]

In [84]:
labels[:5]

['9', '0', '0', '3', '0']

In [85]:
def read_decode_img(img_path):
    """读取解析jpg图片"""
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=1)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img
    

In [86]:
def make_datasets(data_dir, train=False):
    if train:
        images_path = str(data_dir/'fashion_train_jpg_60000/*')
        labels_path = data_dir/'fashion_train_jpg_60000.txt'
    else:
        images_path = str(data_dir/'fashion_test_jpg_10000/*')
        labels_path = data_dir/'fashion_test_jpg_10000.txt'
    image_path_ds = tf.data.Dataset.list_files(images_path)   
    image_ds = image_path_ds.map(read_decode_img, num_parallel_calls=tf.data.experimental.AUTOTUNE) 
    label_info = (labels_path).open(encoding='utf-8').readlines()
    labels = [int(item.split()[-1]) for item in label_info]
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))
    return image_label_ds

In [87]:
BATCH_SIZE = 32
train_ds = make_datasets(data_dir, train=True).repeat().shuffle(num_example).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
test_ds = make_datasets(data_dir).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [95]:

image_path_ds = tf.data.Dataset.list_files(str(data_dir/'fashion_train_jpg_60000/*')) 
for i in image_path_ds.take(5):
    print(i)

tf.Tensor(b'/home/share/fashion_image_label/fashion_train_jpg_60000/10841_9.jpg', shape=(), dtype=string)
tf.Tensor(b'/home/share/fashion_image_label/fashion_train_jpg_60000/41774_1.jpg', shape=(), dtype=string)
tf.Tensor(b'/home/share/fashion_image_label/fashion_train_jpg_60000/49797_1.jpg', shape=(), dtype=string)
tf.Tensor(b'/home/share/fashion_image_label/fashion_train_jpg_60000/45171_4.jpg', shape=(), dtype=string)
tf.Tensor(b'/home/share/fashion_image_label/fashion_train_jpg_60000/40527_7.jpg', shape=(), dtype=string)


In [ ]:
for img, label in test_ds.take(1):
    print(img.numpy().shape)
    print(label)

In [91]:
model = keras.models.Sequential()
# 经过3次卷积(2层3*3的卷积核)池化(1层)操作
model.add(keras.layers.Conv2D(filters=32, kernel_size=5, padding='same',
                              activation='relu', input_shape=(28, 28, 1)))
model.add(keras.layers.Conv2D(filters=32, kernel_size=5, padding='same', activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=2))
# model.add(keras.layers.BatchNormalization())
# model.add(keras.layers.Activation('selu'))

model.add(keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=2))
# model.add(keras.layers.BatchNormalization())
# model.add(keras.layers.Activation('selu'))

model.add(keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=2))
# model.add(keras.layers.BatchNormalization())
# model.add(keras.layers.Activation('selu'))
          
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.AlphaDropout(rate=0.5))

model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['sparse_categorical_accuracy'])

In [92]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 7, 7, 128)        

In [93]:
callbacks = [
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3)
]
history = model.fit(train_ds, epochs=10,
                    validation_data=test_ds,
                    steps_per_epoch=num_train_sample // BATCH_SIZE,
                    validation_steps = num_test_sample // BATCH_SIZE,
                    callbacks=callbacks)

Epoch 1/10
1401/1875 [=====================>........] - ETA: 1:07 - loss: 2.5334 - sparse_categorical_accuracy: 0.0999

KeyboardInterrupt: 